# FESTA Experiment on Google Colab - Full Dataset

**Processing 100 samples per task (300 total samples)**

This notebook runs the FESTA (Feature-Equivalent and Semantically-Targeted Augmentation) experiment on the TREA dataset stored in Google Drive.

## Features:
- ✅ Dataset stored in Google Drive (upload once, use always)
- ✅ Checkpoint/resume capability (survives session timeouts)
- ✅ Results saved to Drive automatically
- ✅ GPU memory optimization
- ✅ Progress tracking and monitoring

## Expected Runtime:
- **Total samples**: 300 (100 per task)
- **Estimated time**: 5-8 hours on Colab GPU
- **Per sample**: ~1-2 minutes

## Prerequisites:
1. TREA_dataset folder uploaded to Google Drive root (`My Drive/TREA_dataset`)
2. Colab with GPU runtime (Runtime > Change runtime type > GPU)

---
## 1. Mount Google Drive

This will prompt you to authorize access to your Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted successfully!")

---
## 2. Verify GPU and System Info

In [ ]:
!nvidia-smi

import torch
print(f"\n{'='*60}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"{'='*60}")

---
## 3. Verify Dataset in Google Drive

In [ ]:
import os
from pathlib import Path

# Check if dataset exists in Drive
dataset_path = Path("/content/drive/MyDrive/TREA_dataset")

if not dataset_path.exists():
    print("❌ ERROR: TREA_dataset folder not found in Google Drive!")
    print("\n📋 Please follow these steps:")
    print("   1. Upload the TREA_dataset folder to 'My Drive' (root level)")
    print("   2. Wait for upload to complete")
    print("   3. Re-run this cell")
    raise FileNotFoundError("Dataset not found in Google Drive")

print("✅ Dataset found in Google Drive!")
print(f"\n📁 Dataset location: {dataset_path}")
print("\n📊 Dataset structure:")

# Check each task folder
for task in ["count", "order", "duration"]:
    task_dir = dataset_path / task
    csv_file = task_dir / f"{task}.csv"
    
    if csv_file.exists():
        # Count lines (samples)
        with open(csv_file, 'r') as f:
            num_samples = len(f.readlines()) - 1  # -1 for header
        print(f"  ✅ {task}: {num_samples} samples")
    else:
        print(f"  ❌ {task}: CSV file not found")

print("\n✅ All dataset files verified!")

---
## 4. Clone Repository and Install Dependencies

In [ ]:
# Clone the repository
!git clone https://github.com/atinmathur/AudioLLM-FESTA.git /content/AudioLLM-FESTA

%cd /content/AudioLLM-FESTA

print("✅ Repository ready!")

In [ ]:
# Install dependencies
!pip install -q transformers>=4.37.0
!pip install -q torch torchaudio
!pip install -q librosa soundfile
!pip install -q pandas numpy scikit-learn
!pip install -q pyyaml tqdm
!pip install -q qwen-audio-chat  # Qwen2-Audio specific

print("✅ All dependencies installed!")

---
## 5. Copy Config to Repository

Copy the full dataset config file to the repository directory.

In [ ]:
# If config_colab_full.yaml is in the repo, it's already there
# Otherwise, create it here:

import yaml

config = {
    'model': {
        'name': 'Qwen/Qwen2-Audio-7B-Instruct',
        'device': 'cuda',
        'dtype': 'float16',
        'max_length': 512
    },
    'dataset': {
        'data_dir': '/content/drive/MyDrive/TREA_dataset',
        'tasks': ['count', 'order', 'duration'],
        'samples_per_task': 100,
        'random_seed': 42
    },
    'festa': {
        'n_fes_audio': 5,
        'n_fes_text': 2,
        'n_fcs_audio': 5,
        'n_fcs_text': 2
    },
    'fes_transforms': {
        'audio': [
            {'type': 'add_silence', 'params': {'duration_range': [0.1, 0.3], 'position': 'between'}},
            {'type': 'volume_adjustment', 'params': {'gain_range': [-0.15, 0.15]}},
            {'type': 'add_noise', 'params': {'snr_range': [30, 40]}}
        ],
        'text': {'paraphrase_method': 'manual', 'num_paraphrases': 2}
    },
    'fcs_transforms': {
        'count': {'audio': [{'type': 'add_event', 'params': {'source': 'synthetic_silences', 'position': ['start', 'end']}}]},
        'order': {'audio': [{'type': 'swap_events', 'params': {'mode': 'adjacent'}}]},
        'duration': {'audio': [{'type': 'replace_event', 'params': {'target': ['longest', 'shortest']}}]},
        'text': {'reverse_relationships': True, 'avoid_negation': True}
    },
    'baselines': {
        'output_entropy': {'enabled': True, 'temperature': 1.0, 'num_samples': 10},
        'verbalized_confidence': {'enabled': False},
        'input_augmentation': {'enabled': False},
        'rephrase_uncertainty': {'enabled': True, 'num_rephrases': 3},
        'blackbox_uncertainty': {'enabled': False}
    },
    'metrics': ['auroc', 'accuracy', 'selective_risk', 'coverage_accuracy'],
    'experiment': {
        'output_dir': '/content/drive/MyDrive/festa_results',
        'save_predictions': True,
        'save_uncertainty_scores': True,
        'save_intermediate': True,
        'checkpoint_file': '/content/drive/MyDrive/festa_checkpoint.json',
        'log_level': 'INFO'
    },
    'hardware': {
        'batch_size': 1,
        'num_workers': 2,
        'pin_memory': True,
        'clear_cache': True
    },
    'colab': {
        'auto_save_interval': 1,
        'memory_monitor': True,
        'test_mode': False
    }
}

# Save config
with open('config_colab_full.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("✅ Config file created: config_colab_full.yaml")
print(f"   - Dataset: 100 samples per task (300 total)")
print(f"   - Results will be saved to: /content/drive/MyDrive/festa_results")
print(f"   - Checkpoint: /content/drive/MyDrive/festa_checkpoint.json")

---
## 6. Check Existing Progress (Resume Capability)

In [ ]:
import json
from pathlib import Path

checkpoint_file = Path("/content/drive/MyDrive/festa_checkpoint.json")

if checkpoint_file.exists():
    with open(checkpoint_file, 'r') as f:
        checkpoint = json.load(f)
    
    completed = len(checkpoint.get('completed_sample_ids', []))
    total = checkpoint.get('total_samples', 300)
    progress = checkpoint.get('progress_percent', 0)
    
    print("📂 Found existing checkpoint!")
    print(f"   Progress: {completed}/{total} samples ({progress:.1f}%)")
    print(f"   Last updated: {checkpoint.get('last_updated', 'Unknown')}")
    print("\n✅ Experiment will resume from where it left off")
else:
    print("📝 No checkpoint found - starting fresh")
    print("   Will process all 300 samples (100 per task)")

---
## 7. Run FESTA Experiment

**⚠️ IMPORTANT:**
- This will take **5-8 hours** to complete
- Progress is saved after each sample
- If Colab disconnects, just re-run this cell to resume
- Results are saved to Google Drive automatically

In [ ]:
# Run the experiment
!python experiments/run_festa_colab.py --config config_colab_full.yaml

---
## 8. Monitor Progress (Run this in a separate cell while experiment is running)

In [ ]:
import json
import time
from pathlib import Path
from IPython.display import clear_output

checkpoint_file = Path("/content/drive/MyDrive/festa_checkpoint.json")

# Monitor for 60 iterations (5 minutes if checking every 5 seconds)
for i in range(60):
    clear_output(wait=True)
    
    if checkpoint_file.exists():
        with open(checkpoint_file, 'r') as f:
            checkpoint = json.load(f)
        
        completed = len(checkpoint.get('completed_sample_ids', []))
        total = checkpoint.get('total_samples', 300)
        progress = checkpoint.get('progress_percent', 0)
        
        print(f"{'='*60}")
        print(f"🔄 FESTA Experiment Progress")
        print(f"{'='*60}")
        print(f"📊 Samples: {completed}/{total} ({progress:.1f}%)")
        print(f"⏱️  Last updated: {checkpoint.get('last_updated', 'Unknown')}")
        
        # Progress bar
        bar_length = 40
        filled = int(bar_length * completed / total)
        bar = '█' * filled + '░' * (bar_length - filled)
        print(f"\n[{bar}]")
        
        if completed == total:
            print("\n🎉 Experiment completed!")
            break
    else:
        print("⏳ Waiting for experiment to start...")
    
    time.sleep(5)  # Check every 5 seconds

---
## 9. View Results

In [ ]:
import json
from pathlib import Path
import glob

results_dir = Path("/content/drive/MyDrive/festa_results")

if not results_dir.exists():
    print("❌ Results directory not found. Run the experiment first.")
else:
    print(f"📁 Results directory: {results_dir}\n")
    
    # List all result files
    result_files = list(results_dir.glob("*.json"))
    
    if not result_files:
        print("⏳ No results yet. Experiment still running...")
    else:
        print(f"📊 Found {len(result_files)} result files:\n")
        
        for f in sorted(result_files):
            print(f"  • {f.name}")
        
        # Try to load latest metrics
        metrics_files = sorted(results_dir.glob("metrics_*.json"))
        if metrics_files:
            latest_metrics = metrics_files[-1]
            with open(latest_metrics, 'r') as f:
                metrics = json.load(f)
            
            print(f"\n{'='*60}")
            print(f"📈 Latest Results")
            print(f"{'='*60}")
            
            if 'overall_accuracy' in metrics:
                print(f"Overall Accuracy: {metrics['overall_accuracy']:.2%}")
            
            if 'method_results' in metrics:
                print("\nMethod AUROC Scores:")
                for method, results in metrics['method_results'].items():
                    auroc = results.get('auroc', 'N/A')
                    if isinstance(auroc, float):
                        print(f"  {method}: {auroc:.4f}")
                    else:
                        print(f"  {method}: {auroc}")

---
## 10. Download Results (Optional)

Results are already in Google Drive, but you can download them to your local machine if needed.

In [ ]:
from google.colab import files
import shutil
from pathlib import Path

results_dir = Path("/content/drive/MyDrive/festa_results")

if results_dir.exists():
    # Create a zip file
    zip_path = "/content/festa_results.zip"
    shutil.make_archive("/content/festa_results", 'zip', results_dir)
    
    print("📦 Created zip file of results")
    print("⬇️  Downloading...")
    
    files.download(zip_path)
    
    print("✅ Download complete!")
else:
    print("❌ No results to download yet")

---
## 11. Resume from Checkpoint (If Session Disconnected)

If your Colab session times out or disconnects:

1. **Re-run cells 1-6** (Mount Drive, verify dataset, install dependencies)
2. **Check cell 6** to see your progress
3. **Re-run cell 7** - it will automatically resume from the last checkpoint

The checkpoint system ensures no work is lost!

---
## Notes

### File Locations:
- **Dataset**: `/content/drive/MyDrive/TREA_dataset`
- **Results**: `/content/drive/MyDrive/festa_results`
- **Checkpoint**: `/content/drive/MyDrive/festa_checkpoint.json`

### Troubleshooting:
- **Out of memory**: The code already clears GPU cache between samples. If still OOM, reduce `n_fes_audio` or `n_fcs_audio` in config.
- **Dataset not found**: Make sure TREA_dataset is in the root of 'My Drive', not in a subfolder
- **Slow progress**: Each sample takes 1-2 minutes. Total runtime: 5-8 hours for 300 samples.

### Monitoring:
- Check GPU usage: Run cell 2 anytime
- Monitor progress: Run cell 8 while experiment runs
- View checkpoint: Run cell 6 anytime